<a href="https://colab.research.google.com/github/Neyder2502/salud-mental-analisis-datos/blob/main/notebooks/Analizis_de_salud_mental_de_los_pelaitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CODIGO HECHO POR:
JOSHUA ALEJANDRO SAAVEDRA CARDENAS: 2252148
ANGEL SANTIAGO BARAJAS ARBELAEZ: 2251511

# **IMPORTAR LIBRERIAS**

In [11]:
import pandas as pd # para trabajar con tablas de datos
import numpy as np # para calculos matematicos y operaciones con numeros
import matplotlib.pyplot as plt
from google.colab import files # herramientas especificas de Google Colab
import io # para manejar archivos en memoria sin guardarlos en disco
import plotly.express as px # para graficos interactivos
import plotly.graph_objects as go # graficos mas personalizados y complejos

# **SUBIR ARCHIVO**

In [12]:
uploaded = files.upload() # abre la ventana para seleccionar un archivo local
file_name = list(uploaded.keys())[0] # toma el nombre del archivo subido
df = pd.read_csv(io.BytesIO(uploaded[file_name])) # convierte el CSV a DataFrame de pandas para analizarlo

Saving Student Mental health.csv to Student Mental health (1).csv


# **PREPROCESAMIENTO**

In [13]:
df.columns = df.columns.str.strip().str.replace(" ", "_") # quito espacios y uso guiones bajos _ para que sea mas facil para el codigo

yes_no_cols = []
for col in df.columns:
    col_values = df[col].dropna().astype(str).str.lower()
    if col_values.isin(["yes", "no"]).all():
        yes_no_cols.append(col)
        # busco colomnas que solo tengan yes/no para analisis binario esto me serive para graficos de conteo y porcentajes

# **LIMPIEZA**

In [14]:
# para el analisis da datos no podemos tener valores vacios, hay que rellenarlos
for col in df.select_dtypes(include=[np.number]):
    df[col].fillna(df[col].median(), inplace=True)
# para los numeros uso la mediana (el valor del medio ordenado), la mediana es mejor que el promedio cuando hay valores extremos
for col in df.select_dtypes(include=["object"]):
    df[col].fillna(df[col].mode()[0], inplace=True)
# para el texto uso la moda, el valor que mas se repite. Si la mayoria son "No" los vacios se covierten en "No"

/tmp/ipython-input-3706616055.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/tmp/ipython-input-3706616055.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [15]:
df.head()

,Timestamp,Choose_your_gender,Age,What_is_your_course?,Your_current_year_of_Study,What_is_your_CGPA?,Marital_status,Do_you_have_Depression?,Do_you_have_Anxiety?,Do_you_have_Panic_attack?,Did_you_seek_any_specialist_for_a_treatment?
0,8/7/2020 12:02,Female,18.0,Engineering,year 1,3.00 - 3.49,No,Yes,No,Yes,No
1,8/7/2020 12:04,Male,21.0,Islamic education,year 2,3.00 - 3.49,No,No,Yes,No,No
2,8/7/2020 12:05,Male,19.0,BIT,Year 1,3.00 - 3.49,No,Yes,Yes,Yes,No
3,8/7/2020 12:06,Female,22.0,Laws,year 3,3.00 - 3.49,Yes,Yes,No,No,No
4,8/7/2020 12:13,Male,23.0,Mathemathics,year 4,3.00 - 3.49,No,No,No,No,No


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 11 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Timestamp                                     101 non-null    object 
 1   Choose_your_gender                            101 non-null    object 
 2   Age                                           101 non-null    float64
 3   What_is_your_course?                          101 non-null    object 
 4   Your_current_year_of_Study                    101 non-null    object 
 5   What_is_your_CGPA?                            101 non-null    object 
 6   Marital_status                                101 non-null    object 
 7   Do_you_have_Depression?                       101 non-null    object 
 8   Do_you_have_Anxiety?                          101 non-null    object 
 9   Do_you_have_Panic_attack?                     101 non-null    obj

In [17]:
df.columns

Index(['Timestamp', 'Choose_your_gender', 'Age', 'What_is_your_course?',
       'Your_current_year_of_Study', 'What_is_your_CGPA?', 'Marital_status',
       'Do_you_have_Depression?', 'Do_you_have_Anxiety?',
       'Do_you_have_Panic_attack?',
       'Did_you_seek_any_specialist_for_a_treatment?'],
      dtype='object')

In [18]:
df.isna().sum()

,0
Timestamp,0
Choose_your_gender,0
Age,0
What_is_your_course?,0
Your_current_year_of_Study,0
What_is_your_CGPA?,0
Marital_status,0
Do_you_have_Depression?,0
Do_you_have_Anxiety?,0
Do_you_have_Panic_attack?,0


In [19]:
df['Age'].value_counts()

,count
Age,
18.0,32
24.0,23
19.0,22
23.0,13
20.0,6
21.0,3
22.0,2


In [20]:
df['Choose_your_gender'].value_counts()

,count
Choose_your_gender,
Female,75
Male,26


# **RESUMEN ESTADISTICO**

In [21]:
print("\n============================")
print("RESUMEN ESTADÍSTICO")
print("============================\n")

display(df.describe(include="all").round(3))
# muestra el conteo, promedios, valores unicos. frecuencia, etc. para numeros el mean, std, min, max, percentiles. para texto los valores mas frecuentes, cantidad de unicos.


RESUMEN ESTADÍSTICO



,Timestamp,Choose_your_gender,Age,What_is_your_course?,Your_current_year_of_Study,What_is_your_CGPA?,Marital_status,Do_you_have_Depression?,Do_you_have_Anxiety?,Do_you_have_Panic_attack?,Did_you_seek_any_specialist_for_a_treatment?
count,101,101,101.000,101,101,101,101,101,101,101,101
unique,92,2,NaN,49,7,6,2,2,2,2,2
top,8/7/2020 12:39,Female,NaN,BCS,year 1,3.50 - 4.00,No,No,No,No,No
freq,3,75,NaN,18,41,47,85,66,67,68,95
mean,NaN,NaN,20.515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,2.488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,18.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,18.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,19.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,23.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# **GRAFICOS**

In [23]:
print("\n============================")
print("DISTRIBUCION DE EDAD Y VARIABLES BINARIAS")
print("============================\n")

age_col = 'Age' # definimos age_col
binary_cols = yes_no_cols # definimos las columnas yes/no en binarios para generar el grafico

# aqui plotly crea graficos
if age_col:
    fig = px.histogram(df, x=age_col, nbins=10, title="Distribución de Edad (Interactivo)")
    fig.show()
# distribucion de estudiantes por edad
for col in binary_cols:
    fig = px.bar(df[col].value_counts(), title=f"Conteo de {col} (Interactivo)")
    fig.show()
# muestra ciantos tienen depreseion, andiedad, ataques de panico, etc.


DISTRIBUCION DE EDAD Y VARIABLES BINARIAS

